# TaXGBoost_Example.ipynb

This notebook is an edited version (forked, see below) of a notebooks that uses Rapids and XGBoost for model training and prediction using the NYC taxi database. The basic flow of the experiment is from [this Dell whitepaper on Rapids performance on different configurations of Dell Servers](https://1drv.ms/b/s!Am_o6IxX4xnWv1s-0m-Rra-K0Ywt) (provided by Bhavesh Patel of Dell).

Ultimately, the purpose of the edits are to create a version of this notebook for use within the OneAPI development (specifically on the Intel DevCloud) including:

- Comparing XGBoost Performance on Intel and Nvidia Configurations
    - Intel Platforms [Devcloud hardware found by console command in ~Sept of 2021, accessible here](https://1drv.ms/x/s!Am_o6IxX4xnWwVpPvh-VGGXQim8Y?e=peFxsn)
        - Devcloud (by proccessor type, most are 1 or 2 GPU/Node)
            - CPU
                - i9-10920x
                - e-2176g
                - clx
                - gold6128
            - GPU
                - "gen 9" (Presumably an iGPU)
                - Iris Xe max (Presumably an dGPU, though Bhavesh has mentioned that this is a lower/mobile version)
        - Dell Server
            - Bhaves should get access to a Intel GPU server with a better GPU this December (2021)
                - This is especially advantageous, since Devcloud dGPU nodes are very memory limited (32 gb)
    - Nvidia Platforms (by node type)
        - HiPerGator [Info here](https://www.rc.ufl.edu/services/hipergator/)
            - AI NVidia DGX A100 SuperPod
                - 140 NVIDIA DGX A100 Nodes
                    - 2x AMD EPYC 7742 (Rome) 64-Core processors with Simultaneous Multi-Threading (SMT) enabled presenting 256 cores per node
                    - 2TB RAM
                    - 8x NVIDIA A100 80GB Tensor Core GPUs
                    - NVSWITCH technology that supports integration of all 8 A100 GPU’s in a system with unified memory space
                    - 10x HDR IB non-blocking interfaces for inter-node communication
                    - 2x 100GbE ethernet interfaces
                    - 28TB NVMe local storage
            - HiperGator 3.0 (Jan 2021)
                - Intel Xeon E5-2698 v3 Haswell, 4GB/core (could possibly use Intel4py versions as a comparison)
            - HiPerGator 3.0 (Same pase, but Q2 2021)
                - AMD EPYX 7702 Rome, 8 GB/core
            - HiperGator 2.0 (2015)
                - AMD EPYC 75F3 Milan, 8GB/core
        - Dell Servers (Used for the whitepaper, and Bhavesh is currently in the process of getting us access to these nodes or similar as well)
            -
            -
            -
            -
            
    


## Sources where the code came from:
-  [First version of this file came from this file/repo (NYCTaxi-E2E.ipynb/rapids notebooks-contrib)](https://github.com/rapidsai-community/notebooks-contrib/blob/main/community_tutorials_and_guides/taxi/NYCTaxi-E2E.ipynb)
-  [This link contains a similar (maybe forked) version of this notebook that I used for reference](https://jovian.ai/allenkong221/nyc-taxi-fare/v/1?utm_source=embed#C2)

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import matplotlib
from scipy import stats
from scipy.stats import norm, skew
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
#import lightgbm as lgbm
import xgboost as xgb

In [2]:
'''if you get 'ModuleNotFoundError: No module named 'gcsfs', run `!pip install gcsfs` 
'''
#base_path = 'gcs://anaconda-public-data/nyc-taxi/csv/'

#df_2014 = dask_cudf.read_csv(base_path+'2014/yellow_*.csv')


#Ok, we need to load in data here, but not the old way
data_path = '/home/u117430/rapids/data/'
#df_2014 = pd.read_csv(data_path+'2014/yellow_*.csv')
df_2014 = pd.read_csv(data_path+'yellow_tripdata_2014-11.csv')
#df_2015 = pd.read_csv(data_path+'yellow_tripdata_2015-11.csv')
df_2016 = pd.read_csv(data_path+'yellow_tripdata_2016-11.csv')
    #Sources:
        #https://github.com/oneapi-src/oneAPI-samples/blob/master/AI-and-Analytics/End-to-end-Workloads/Census/census_modin.ipynb
        #:https://examples.dask.org/dataframes/01-data-access.html#Read-CSV-files

/opt/intel/inteloneapi/intelpython/latest/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Dictionary of required columns and their datatypes
must_haves = {
     ' pickup_datetime': 'datetime64[s]',
     ' dropoff_datetime': 'datetime64[s]',
     ' passenger_count': 'int32',
     ' trip_distance': 'float32',
     ' pickup_longitude': 'float32',
     ' pickup_latitude': 'float32',
     ' rate_code': 'int32',
     ' dropoff_longitude': 'float32',
     ' dropoff_latitude': 'float32',
     ' fare_amount': 'float32'
    }

In [4]:
def clean(ddf, must_haves):
    tmp = {col:col.strip().lower() for col in list(ddf.columns)}     # replace the extraneous spaces in column names and lower the font type
        #In this case, what is tmp? It looks like tmp is jit dictionary built to hold the column names that have been fed in, but stripped of spaces and lower cased
    ddf = ddf.rename(columns=tmp) #Then, this dictionary is used to rename the columns
        
        #Rename documentionation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

    ddf = ddf.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'tpep_dropoff_datetime': 'dropoff_datetime',
        'ratecodeid': 'rate_code'
    })  #More name changing. Just changing column names to an easier to read format
    
    ddf['pickup_datetime'] = ddf['pickup_datetime'].astype('datetime64[ms]')       #Looks to just recast datatype to a date/time format
    ddf['dropoff_datetime'] = ddf['dropoff_datetime'].astype('datetime64[ms]')

        #Astype doc: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html
    
    #Here's where things get tricky. Let's look at df.map_partitions() vs df.apply()
    
        #DataFrame.map_partitions(func, *args, **kwargs)
            #Desc: Apply Python function on each DataFrame partition.
            #Doc: https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.map_partitions.html#dask.dataframe.DataFrame.map_partitions
        
        #DataFrame.apply(func, axis=0, raw=False, result_type=None, args=(), **kwargs)
            #Desc: Apply a function along an axis of the DataFrame.

        #So apply may not be what we want. map_partitions works on partitions, while apply works on axis
            #FYI: apply apparently shouldn't be used b/c it's horribly inefficient
            #DASK dataframes are made up of partitions, which are pandas dataframes?
                #https://docs.dask.org/en/latest/dataframe-best-practices.html
                #https://docs.dask.org/en/latest/dataframe-design.html#dataframe-design-partitions

    
    for col in ddf.columns:                                   #For each column
        if col not in must_haves:                             #If the column isn't in the dictionary
            ddf = ddf.drop(columns=col)                       #Remove it
            continue
        # if column was read as a string, recast as float
        if ddf[col].dtype == 'object':                        #If the column was a string
            ddf[col] = ddf[col].str.fillna('-1')              #Turn it into a float with these two lines
            ddf[col] = ddf[col].astype('float32')
        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
            if 'int' in str(ddf[col].dtype):                 #Convert int's to 32 bit ints
                ddf[col] = ddf[col].astype('int32')
            if 'float' in str(ddf[col].dtype):               #Convert doubles to floats
                ddf[col] = ddf[col].astype('float32')
            ddf[col] = ddf[col].fillna(-1)
    
    return ddf

In [5]:
#df_2014 = df_2014.map_partitions(clean, must_haves, meta=must_haves)
taxi_df = df_2014
tmp = {col:col.strip().lower() for col in list(taxi_df.columns)}
taxi_df = taxi_df.rename(columns=tmp) #Then, this dictionary is used to rename the columns

taxi_df = taxi_df.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'tpep_dropoff_datetime': 'dropoff_datetime',
        'ratecodeid': 'rate_code'
    }) 

taxi_df['pickup_datetime'] = taxi_df['pickup_datetime'].astype('datetime64[ms]')       #Looks to just recast datatype to a date/time format
taxi_df['dropoff_datetime'] = taxi_df['dropoff_datetime'].astype('datetime64[ms]')


In [6]:
taxi_df.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2014-11-01 00:47:00,2014-11-01 01:00:00,5,2.71,-73.960960,40.818780,1,NaN,-73.923620,40.808400,CRD,11.5,0.5,0.5,2.40,0.0,14.90
1,VTS,2014-11-01 00:28:00,2014-11-01 01:00:00,5,4.67,-73.990148,40.729315,1,NaN,-73.954933,40.769335,CRD,23.5,0.5,0.5,6.00,0.0,30.50
2,VTS,2014-11-01 00:39:00,2014-11-01 01:00:00,2,4.52,-73.980122,40.742847,1,NaN,-73.977007,40.784257,CRD,17.0,0.5,0.5,4.38,0.0,22.38
3,VTS,2014-11-01 00:46:00,2014-11-01 00:56:00,1,1.70,-73.932773,40.703382,1,NaN,-73.951693,40.691347,CRD,8.5,0.5,0.5,2.25,0.0,11.75
4,VTS,2014-11-01 00:06:00,2014-11-01 00:59:00,3,8.18,-73.997955,40.729407,1,NaN,-73.884077,40.748452,CSH,37.5,0.5,0.5,0.00,0.0,38.50


In [7]:
#taxi_df = dask.dataframe.multi.concat([df_2014, df_2015, df_2016])
#taxi_df = pd.concat([df_2014, df_2016])
#taxi_df = df_2014

In [8]:
#taxi_df = taxi_df.persist()

In [9]:
final_taxi_df = taxi_df.drop(['vendor_id', 'store_and_fwd_flag', 'payment_type'], axis=1)

In [10]:
#since we calculated the h_distance let's drop the trip_distance column, and then do model training with XGB.
#taxi_df = taxi_df.drop('trip_distance', axis=1)

In [11]:
final_taxi_df.head()

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,2014-11-01 00:47:00,2014-11-01 01:00:00,5,2.71,-73.960960,40.818780,1,-73.923620,40.808400,11.5,0.5,0.5,2.40,0.0,14.90
1,2014-11-01 00:28:00,2014-11-01 01:00:00,5,4.67,-73.990148,40.729315,1,-73.954933,40.769335,23.5,0.5,0.5,6.00,0.0,30.50
2,2014-11-01 00:39:00,2014-11-01 01:00:00,2,4.52,-73.980122,40.742847,1,-73.977007,40.784257,17.0,0.5,0.5,4.38,0.0,22.38
3,2014-11-01 00:46:00,2014-11-01 00:56:00,1,1.70,-73.932773,40.703382,1,-73.951693,40.691347,8.5,0.5,0.5,2.25,0.0,11.75
4,2014-11-01 00:06:00,2014-11-01 00:59:00,3,8.18,-73.997955,40.729407,1,-73.884077,40.748452,37.5,0.5,0.5,0.00,0.0,38.50


In [12]:
## add features

taxi_df['hour'] = taxi_df['pickup_datetime'].dt.hour
taxi_df['year'] = taxi_df['pickup_datetime'].dt.year
taxi_df['month'] = taxi_df['pickup_datetime'].dt.month
taxi_df['day'] = taxi_df['pickup_datetime'].dt.day
taxi_df['day_of_week'] = taxi_df['pickup_datetime'].dt.weekday
taxi_df['is_weekend'] = (taxi_df['day_of_week']>=5).astype('int32')

#calculate the time difference between dropoff and pickup.
taxi_df['diff'] = taxi_df['dropoff_datetime'].astype('int64') - taxi_df['pickup_datetime'].astype('int64')
taxi_df['diff']=(taxi_df['diff']/1000).astype('int64')

taxi_df['pickup_latitude_r'] = taxi_df['pickup_latitude']//.01*.01
taxi_df['pickup_longitude_r'] = taxi_df['pickup_longitude']//.01*.01
taxi_df['dropoff_latitude_r'] = taxi_df['dropoff_latitude']//.01*.01
taxi_df['dropoff_longitude_r'] = taxi_df['dropoff_longitude']//.01*.01

#taxi_df = taxi_df.drop('pickup_datetime', axis=1)
#taxi_df = taxi_df.drop('dropoff_datetime', axis=1)

In [13]:
for col in taxi_df.columns:
    print(col)

vendor_id
pickup_datetime
dropoff_datetime
passenger_count
trip_distance
pickup_longitude
pickup_latitude
rate_code
store_and_fwd_flag
dropoff_longitude
dropoff_latitude
payment_type
fare_amount
surcharge
mta_tax
tip_amount
tolls_amount
total_amount
hour
year
month
day
day_of_week
is_weekend
diff
pickup_latitude_r
pickup_longitude_r
dropoff_latitude_r
dropoff_longitude_r


In [14]:
final_taxi_df = taxi_df.drop(['pickup_datetime','dropoff_datetime','vendor_id', 'store_and_fwd_flag', 'payment_type'], axis=1)

In [15]:
X, y = final_taxi_df.drop('fare_amount', axis = 1), final_taxi_df['fare_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [16]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [17]:
dvalid = xgb.DMatrix(X_test, label=y_test)

In [18]:
dtest = xgb.DMatrix(final_taxi_df)

In [19]:
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [20]:
xgb_params = {
    'min_child_weight': 1, 
    'learning_rate': 0.05, 
    'colsample_bytree': 0.7, 
    'max_depth': 10,
    'subsample': 0.7,
    'n_estimators': 5000,
    'n_jobs': -1, 
    'booster' : 'gbtree', 
    'silent': 1,
    'eval_metric': 'rmse'}

In [21]:
model = xgb.train(xgb_params, dtrain, 700, watchlist, early_stopping_rounds=100, maximize=False, verbose_eval=50)

[06:44:10] WARNING: /localdisk/tools/tc/agent2/work/30ccf72606c6a49e/base/conda-bld/xgboost_1623788730212/work/src/learner.cc:574: 
Parameters: { "n_estimators", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:15.40109	valid-rmse:15.38414


KeyboardInterrupt: 

In [ ]:
y_train_pred = model.predict(dtrain)
y_pred = model.predict(dvalid)
print('Train r2 score: ', r2_score(y_train_pred, y_train))
print('Test r2 score: ', r2_score(y_test, y_pred))
train_rmse = np.sqrt(mean_squared_error(y_train_pred, y_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Train RMSE: {train_rmse:.4f}')
print(f'Test RMSE: {test_rmse:.4f}')